# Feature Engineering

## Import and Reading Data

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import zscore

In [2]:
df = pd.read_csv("../0_Datasets/IoMT_AI_IDS/Flows/IP-Based Flows Dataset.csv")

C:\Users\ivoafonsobispo\AppData\Local\Temp\ipykernel_7828\3814736681.py:1: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../0_Datasets/IoMT_AI_IDS/Flows/IP-Based Flows Dataset.csv")


In [3]:
df = df[[
#    'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service',
#    'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig',
#    'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
#    'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', 'flow_duration',
    'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot', 'bwd_data_pkts_tot',
    'fwd_pkts_per_sec', 'bwd_pkts_per_sec', 'flow_pkts_per_sec', 'down_up_ratio',
    'fwd_header_size_tot', 'fwd_header_size_min', 'fwd_header_size_max',
    'bwd_header_size_tot', 'bwd_header_size_min', 'bwd_header_size_max',
#    'flow_FIN_flag_count', 'flow_SYN_flag_count', 'flow_RST_flag_count',
#    'fwd_PSH_flag_count', 'bwd_PSH_flag_count', 'flow_ACK_flag_count',
#    'fwd_URG_flag_count', 'bwd_URG_flag_count', 'flow_CWR_flag_count',
#    'flow_ECE_flag_count', 
    'fwd_pkts_payload.min', 'fwd_pkts_payload.max',
    'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg', 'fwd_pkts_payload.std',
    'bwd_pkts_payload.min', 'bwd_pkts_payload.max', 'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg', 'bwd_pkts_payload.std', 'flow_pkts_payload.min',
    'flow_pkts_payload.max', 'flow_pkts_payload.tot', 'flow_pkts_payload.avg','flow_pkts_payload.std', 
#    'fwd_iat.min', 'fwd_iat.max', 'fwd_iat.tot','fwd_iat.avg', 'fwd_iat.std', 
#    'bwd_iat.min', 'bwd_iat.max', 'bwd_iat.tot','bwd_iat.avg', 'bwd_iat.std', 
#    'flow_iat.min', 'flow_iat.max', 'flow_iat.tot','flow_iat.avg', 'flow_iat.std', 'payload_bytes_per_second',
    'fwd_subflow_pkts', 'bwd_subflow_pkts', 'fwd_subflow_bytes',
    'bwd_subflow_bytes', 
# 'fwd_bulk_bytes', 'bwd_bulk_bytes', 'fwd_bulk_packets','bwd_bulk_packets', 'fwd_bulk_rate', 'bwd_bulk_rate', 
#    'active.min','active.max', 'active.tot', 'active.avg', 'active.std', 
#    'idle.min','idle.max', 'idle.tot', 'idle.avg', 'idle.std', 
#    'fwd_init_window_size','bwd_init_window_size', 'fwd_last_window_size', 'bwd_last_window_size',
    'traffic', 'is_attack']].copy()

In [4]:
features = [
    'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot', 'bwd_data_pkts_tot',
    'fwd_pkts_per_sec', 'bwd_pkts_per_sec', 'flow_pkts_per_sec', 'down_up_ratio',
    'fwd_header_size_tot', 'fwd_header_size_min', 'fwd_header_size_max',
    'bwd_header_size_tot', 'bwd_header_size_min', 'bwd_header_size_max',
    'fwd_pkts_payload.min', 'fwd_pkts_payload.max',
    'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg', 'fwd_pkts_payload.std',
    'bwd_pkts_payload.min', 'bwd_pkts_payload.max', 'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg', 'bwd_pkts_payload.std', 'flow_pkts_payload.min',
    'flow_pkts_payload.max', 'flow_pkts_payload.tot', 'flow_pkts_payload.avg','flow_pkts_payload.std', 
    'fwd_subflow_pkts', 'bwd_subflow_pkts', 'fwd_subflow_bytes',
    'bwd_subflow_bytes', 'traffic', 'is_attack']

In [5]:
features_wo_target = [
    'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot', 'bwd_data_pkts_tot',
    'fwd_pkts_per_sec', 'bwd_pkts_per_sec', 'flow_pkts_per_sec', 'down_up_ratio',
    'fwd_header_size_tot', 'fwd_header_size_min', 'fwd_header_size_max',
    'bwd_header_size_tot', 'bwd_header_size_min', 'bwd_header_size_max',
    'fwd_pkts_payload.min', 'fwd_pkts_payload.max',
    'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg', 'fwd_pkts_payload.std',
    'bwd_pkts_payload.min', 'bwd_pkts_payload.max', 'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg', 'bwd_pkts_payload.std', 'flow_pkts_payload.min',
    'flow_pkts_payload.max', 'flow_pkts_payload.tot', 'flow_pkts_payload.avg','flow_pkts_payload.std', 
    'fwd_subflow_pkts', 'bwd_subflow_pkts', 'fwd_subflow_bytes',
    'bwd_subflow_bytes']

In [6]:
df.shape

(3243188, 35)

In [7]:
max_width = max(map(len, df.columns))  # Find the maximum width of column names

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col.ljust(max_width)}: {str(dtype)}")

fwd_pkts_tot         : int64
bwd_pkts_tot         : int64
fwd_data_pkts_tot    : int64
bwd_data_pkts_tot    : int64
fwd_pkts_per_sec     : float64
bwd_pkts_per_sec     : float64
flow_pkts_per_sec    : float64
down_up_ratio        : float64
fwd_header_size_tot  : int64
fwd_header_size_min  : int64
fwd_header_size_max  : int64
bwd_header_size_tot  : int64
bwd_header_size_min  : int64
bwd_header_size_max  : int64
fwd_pkts_payload.min : float64
fwd_pkts_payload.max : float64
fwd_pkts_payload.tot : float64
fwd_pkts_payload.avg : float64
fwd_pkts_payload.std : float64
bwd_pkts_payload.min : float64
bwd_pkts_payload.max : float64
bwd_pkts_payload.tot : float64
bwd_pkts_payload.avg : float64
bwd_pkts_payload.std : float64
flow_pkts_payload.min: float64
flow_pkts_payload.max: float64
flow_pkts_payload.tot: float64
flow_pkts_payload.avg: float64
flow_pkts_payload.std: float64
fwd_subflow_pkts     : float64
bwd_subflow_pkts     : float64
fwd_subflow_bytes    : float64
bwd_subflow_bytes    : float

## Remove Duplicates

In [8]:
# Remove duplicates
df = df.loc[~df.duplicated(subset=features)].reset_index(drop=True).copy()

In [9]:
df.shape

(275973, 35)

## One Hot Encoding

In [10]:
def one_hot_encoding(df, columns):
    for col in columns:
        print(f'[ONE HOT ENCODING] {col}')
        df = pd.get_dummies(df, columns=[col], prefix=col)  # Perform one-hot encoding on the column
    
    return df

In [11]:
cols_to_encode = [
    'proto',
    'conn_state',
]

# df = one_hot_encoding(df,cols_to_encode)

## Remove Outliers

In [12]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print(f'[REMOVED OUTLIERS] {col}')
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [13]:
outliers = features_wo_target

df = remove_outliers(df, outliers, 3)

[REMOVED OUTLIERS] fwd_pkts_tot
[REMOVED OUTLIERS] bwd_pkts_tot
[REMOVED OUTLIERS] fwd_data_pkts_tot
[REMOVED OUTLIERS] bwd_data_pkts_tot
[REMOVED OUTLIERS] fwd_pkts_per_sec
[REMOVED OUTLIERS] bwd_pkts_per_sec
[REMOVED OUTLIERS] flow_pkts_per_sec
[REMOVED OUTLIERS] down_up_ratio
[REMOVED OUTLIERS] fwd_header_size_tot
[REMOVED OUTLIERS] fwd_header_size_min
[REMOVED OUTLIERS] fwd_header_size_max
[REMOVED OUTLIERS] bwd_header_size_tot
[REMOVED OUTLIERS] bwd_header_size_min
[REMOVED OUTLIERS] bwd_header_size_max
[REMOVED OUTLIERS] fwd_pkts_payload.min
[REMOVED OUTLIERS] fwd_pkts_payload.max
[REMOVED OUTLIERS] fwd_pkts_payload.tot
[REMOVED OUTLIERS] fwd_pkts_payload.avg
[REMOVED OUTLIERS] fwd_pkts_payload.std
[REMOVED OUTLIERS] bwd_pkts_payload.min
[REMOVED OUTLIERS] bwd_pkts_payload.max
[REMOVED OUTLIERS] bwd_pkts_payload.tot
[REMOVED OUTLIERS] bwd_pkts_payload.avg
[REMOVED OUTLIERS] bwd_pkts_payload.std
[REMOVED OUTLIERS] flow_pkts_payload.min
[REMOVED OUTLIERS] flow_pkts_payload.max
[REM

## Normalize, Z-score

In [14]:
def zscore_normalization(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        print(f"[Z-SCORE] {col}")
        df[col] = zscore(df[col])
    
    print("[DONE] Z-score Normalization")
    print("[INFO] Current Fields in the DataFrame:")
    return df

In [15]:
cols_to_zscore = features_wo_target

df = zscore_normalization(df, cols_to_zscore)

[Z-SCORE] fwd_pkts_tot
[Z-SCORE] bwd_pkts_tot
[Z-SCORE] fwd_data_pkts_tot
[Z-SCORE] bwd_data_pkts_tot
[Z-SCORE] fwd_pkts_per_sec
[Z-SCORE] bwd_pkts_per_sec
[Z-SCORE] flow_pkts_per_sec
[Z-SCORE] down_up_ratio
[Z-SCORE] fwd_header_size_tot
[Z-SCORE] fwd_header_size_min
[Z-SCORE] fwd_header_size_max
[Z-SCORE] bwd_header_size_tot
[Z-SCORE] bwd_header_size_min
[Z-SCORE] bwd_header_size_max
[Z-SCORE] fwd_pkts_payload.min
[Z-SCORE] fwd_pkts_payload.max
[Z-SCORE] fwd_pkts_payload.tot
[Z-SCORE] fwd_pkts_payload.avg
[Z-SCORE] fwd_pkts_payload.std
[Z-SCORE] bwd_pkts_payload.min
[Z-SCORE] bwd_pkts_payload.max
[Z-SCORE] bwd_pkts_payload.tot
[Z-SCORE] bwd_pkts_payload.avg
[Z-SCORE] bwd_pkts_payload.std
[Z-SCORE] flow_pkts_payload.min
[Z-SCORE] flow_pkts_payload.max
[Z-SCORE] flow_pkts_payload.tot
[Z-SCORE] flow_pkts_payload.avg
[Z-SCORE] flow_pkts_payload.std
[Z-SCORE] fwd_subflow_pkts
[Z-SCORE] bwd_subflow_pkts
[Z-SCORE] fwd_subflow_bytes
[Z-SCORE] bwd_subflow_bytes
[DONE] Z-score Normalization
[IN

In [16]:
df

,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,...,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,traffic,is_attack
3,-0.865193,-1.141364,-0.832738,-0.74695,-0.585301,-0.476004,-0.541332,-2.050919,-0.907287,-66.222439,...,-0.508541,-0.823871,-0.491339,-0.531550,-0.792487,-0.984044,-0.800876,-0.992819,arpspoofing,1
8,-0.435985,-1.141364,-0.450908,-0.74695,-0.585301,-0.476004,-0.541332,-2.050919,-0.441117,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.745970,-0.984044,-0.805688,-0.992819,arpspoofing,1
10,-0.674434,-0.621972,-0.450908,-0.74695,-0.572698,-0.467107,-0.530499,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,arpspoofing,1
14,-0.674434,-0.621972,-0.450908,-0.74695,-0.572624,-0.467055,-0.530436,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,arpspoofing,1
19,-0.531365,-1.141364,-0.546365,-0.74695,-0.585301,-0.476004,-0.541332,-2.050919,-0.536742,0.002967,...,-0.517239,-0.824912,-0.557857,-0.527991,-0.745970,-0.984044,-0.805688,-0.992819,arpspoofing,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275961,-0.674434,-0.621972,-0.450908,-0.74695,-0.572629,-0.467058,-0.530440,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,slowread,1
275962,-0.865193,-1.037486,-0.928195,-0.74695,-0.351997,-0.311307,-0.340799,0.232052,-0.871428,0.002967,...,-0.519508,-0.829911,-0.565532,-0.531550,-0.792487,-0.886108,-0.806943,-0.992819,slowread,1
275963,-0.865193,-1.037486,-0.928195,-0.74695,-0.353884,-0.312640,-0.342421,0.232052,-0.871428,0.002967,...,-0.519508,-0.829911,-0.565532,-0.531550,-0.792487,-0.886108,-0.806943,-0.992819,slowread,1
275969,-0.865193,-1.037486,-0.928195,-0.74695,-0.356351,-0.314381,-0.344541,0.232052,-0.871428,0.002967,...,-0.519508,-0.829911,-0.565532,-0.531550,-0.792487,-0.886108,-0.806943,-0.992819,slowread,1


In [17]:
df.shape

(193461, 35)

In [18]:
df.groupby('traffic')['traffic'].count()

traffic
apachekiller    35873
arpspoofing        94
camoverflow        21
mqttmalaria     68866
netscan           575
normal          35162
rudeadyet       37538
slowloris       12208
slowread         3124
Name: traffic, dtype: int64

In [19]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0     35162
1    158299
Name: is_attack, dtype: int64

## Undersampling

In [20]:
undersample_count = 10000
category_to_undersample = 'mqttmalaria'

category_count = df.groupby('traffic')['traffic'].count().loc[category_to_undersample]

undersampled_df = df[df['traffic'] == category_to_undersample].sample(n=undersample_count, random_state=42)
df_undersampled = pd.concat([df[df['traffic'] != category_to_undersample], undersampled_df])
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

df = df_undersampled.copy()

In [21]:
undersample_count = 10000
category_to_undersample = 'apachekiller'

category_count = df.groupby('traffic')['traffic'].count().loc[category_to_undersample]

undersampled_df = df[df['traffic'] == category_to_undersample].sample(n=undersample_count, random_state=42)
df_undersampled = pd.concat([df[df['traffic'] != category_to_undersample], undersampled_df])
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

df = df_undersampled.copy()

In [22]:
undersample_count = 10000
category_to_undersample = 'rudeadyet'

category_count = df.groupby('traffic')['traffic'].count().loc[category_to_undersample]

undersampled_df = df[df['traffic'] == category_to_undersample].sample(n=undersample_count, random_state=42)
df_undersampled = pd.concat([df[df['traffic'] != category_to_undersample], undersampled_df])
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

df = df_undersampled.copy()

In [23]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    35162
1    46022
Name: is_attack, dtype: int64

In [24]:
df.groupby('traffic')['traffic'].count()

traffic
apachekiller    10000
arpspoofing        94
camoverflow        21
mqttmalaria     10000
netscan           575
normal          35162
rudeadyet       10000
slowloris       12208
slowread         3124
Name: traffic, dtype: int64

---------------------------------------

# Train and Test Dataset

In [25]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [26]:
X_columns = df.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df["is_attack"].values)

X = df[X_columns].values
y = df["is_attack"].values
y = le.transform(y)

In [27]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [28]:
X_test.shape,y_test.shape

((20296, 34), (20296,))

In [29]:
X_train.shape,y_train.shape

((60888, 34), (60888,))

In [30]:
# Convert arrays to Series or DataFrame
X_train = pd.DataFrame(X_train, columns=X_columns)
y_train = pd.Series(y_train, name='is_attack')
X_test = pd.DataFrame(X_test, columns=X_columns)
y_test = pd.Series(y_test, name='is_attack')

In [31]:
# Create new DataFrames for training and testing sets
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [32]:
train_df.head(10)

,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,...,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,traffic,is_attack
0,-0.674434,-0.621972,-0.450908,-0.74695,-0.572718,-0.467121,-0.530517,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
1,-0.722124,-0.72585,-0.546365,-0.74695,-0.572239,-0.466783,-0.530104,0.232052,-0.727991,0.002967,...,-0.517239,-0.824912,-0.557857,-0.527991,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
2,-0.674434,-0.621972,-0.450908,-0.74695,-0.572705,-0.467112,-0.530505,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
3,-0.674434,-0.621972,-0.450908,-0.74695,-0.572649,-0.467072,-0.530456,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
4,-0.865193,-0.933607,-0.928195,-0.74695,-0.054334,0.273648,0.143245,2.515023,-0.871428,0.002967,...,-0.519508,-0.829911,-0.565532,-0.53155,-0.792487,-0.788171,-0.806943,-0.992819,rudeadyet,1
5,-0.769814,-0.933607,-0.832738,-0.74695,0.888997,0.21783,0.514675,-0.528937,-0.76385,0.002967,...,-0.398113,-0.763058,-0.401284,-0.372243,-0.699453,-0.788171,-0.739788,-0.992819,slowloris,1
6,-0.674434,-0.621972,-0.450908,-0.74695,-0.572642,-0.467067,-0.530451,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
7,-0.769814,-0.933607,-0.832738,-0.74695,0.901355,0.223646,0.523527,-0.528937,-0.76385,0.002967,...,-0.431393,-0.781385,-0.446311,-0.415916,-0.699453,-0.788171,-0.758198,-0.992819,slowloris,1
8,1.471606,1.455598,1.458239,1.467491,-0.478793,-0.43841,-0.472672,-0.909433,1.471377,0.002967,...,-0.448789,0.124147,-0.409266,-0.438132,1.486861,1.464372,0.130504,0.190744,mqttmalaria,1
9,-0.865193,-0.933607,-0.928195,-0.74695,4.274848,6.385842,5.724858,2.515023,-0.871428,0.002967,...,-0.519508,-0.829911,-0.565532,-0.53155,-0.792487,-0.788171,-0.806943,-0.992819,rudeadyet,1


In [33]:
test_df.head(10)

,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,...,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,traffic,is_attack
0,-0.817503,-0.933607,-0.73728,-0.74695,-0.482514,-0.403444,-0.452983,0.232052,-0.823616,0.002967,...,-0.509298,-0.818664,-0.530994,-0.514251,-0.74597,-0.788171,-0.795646,-0.992819,slowloris,1
1,-0.579055,-0.414215,-0.641823,-0.658372,0.552561,0.327246,0.436698,0.232052,-0.572601,0.002967,...,2.218502,1.967492,1.889052,1.878208,-0.513383,-0.298488,1.971296,0.806197,apachekiller,1
2,-0.674434,-0.621972,-0.450908,-0.74695,-0.572623,-0.467054,-0.530434,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
3,-0.674434,-0.621972,-0.450908,-0.74695,-0.572631,-0.467059,-0.530441,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
4,-0.674434,-0.621972,-0.450908,-0.74695,-0.572633,-0.467061,-0.530443,0.232052,-0.680179,0.002967,...,-0.517239,-0.823663,-0.557857,-0.528041,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
5,1.566985,1.559476,1.553696,1.556069,-0.475965,-0.437412,-0.470848,-0.909433,1.567002,0.002967,...,-0.448789,0.16455,-0.408913,-0.437967,1.579896,1.562309,0.170252,0.238087,mqttmalaria,1
6,-0.722124,-0.72585,-0.546365,-0.74695,-0.571789,-0.466465,-0.529717,0.232052,-0.727991,0.002967,...,-0.517239,-0.824912,-0.557857,-0.527991,-0.792487,-0.886108,-0.805688,-0.992819,normal,0
7,-0.769814,-0.933607,-0.832738,-0.74695,0.877967,0.212639,0.506774,-0.528937,-0.76385,0.002967,...,-0.462025,-0.798254,-0.487757,-0.456115,-0.699453,-0.788171,-0.775144,-0.992819,slowloris,1
8,-0.722124,-0.829729,-0.73728,-0.74695,-0.569616,-0.467699,-0.529535,-0.338691,-0.716038,0.002967,...,-0.435553,-0.77722,-0.473065,-0.439703,-0.74597,-0.837139,-0.780478,-0.992819,slowloris,1
9,-0.626744,-0.518094,-0.73728,-0.658372,0.399673,0.219318,0.305286,0.232052,-0.620413,0.002967,...,2.218502,1.936669,2.266596,2.316445,-0.559901,-0.396424,1.940334,0.806197,apachekiller,1


---------------------------------------

# Save Train and Test Dataset

In [34]:
import os

# Convert arrays to Series or DataFrame
X_train = pd.DataFrame(X_train, columns=X_columns)
y_train = pd.Series(y_train, name='is_attack')
X_test = pd.DataFrame(X_test, columns=X_columns)
y_test = pd.Series(y_test, name='is_attack')

# Create new DataFrames for training and testing sets
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Directory to save the files
save_dir = '../0_Datasets/IoMT_AI_IDS/Flows/'

# Save train and test DataFrames to CSV files in the specified directory
train_path = os.path.join(save_dir, 'train.csv')
test_path = os.path.join(save_dir, 'test.csv')

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)